In [ ]:
#Importing necessary libraries
import hopsworks
import hsfs
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler  # Import StandardScaler from scikit-learn
import joblib

load_dotenv()

#Connecting to hopsworks
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

#Another connection to hopsworks
api_key = os.getenv('hopsworks_api')
connection = hsfs.connection()
fs = connection.get_feature_store()

In [ ]:
#Getting the feature view
feature_view = fs.get_feature_view(
    name='amd_stock_fv',
    version=1
)

In [ ]:
#Setting up train & test split dates (datoer) skal nok ændres i forhold til træning og test for mest optimale perioder!
train_start = "2022-03-10"
train_end = "2023-11-30"

test_start = '2024-02-02'
test_end = "2024-04-07"

In [ ]:
feature_view.create_train_test_split(
    train_start=train_start,
    train_end=train_end,
    test_start=test_start,
    test_end=test_end,
    data_format='csv',
    coalesce= True,
    statistics_config={'histogram':True,'correlations':True})